In [1]:
# Initialize app config
import importlib
from demolib import schema, cfg, spark
from demolib.functions import earth_distance
from demolib.mongo import *
from demolib.udf import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
# Load customer information from source CSV file
# _id column is added to make load job idempotent.
customer_df = spark.read \
    .option('header', True) \
    .csv('{}/{}'.format(cfg.load.dir, cfg.load.customer), schema = schema.customer.schema) \
    .withColumn('_id', expr("cc_num || '+' || first || '+' || last"))

In [3]:
# Add customer age column to customer data
customer_age_df = customer_df.withColumn("age", udf_age())

In [4]:
customer_age_df.show(2)

+----------------+-------+--------+------+--------------------+--------------+-----+-----+-------+---------+--------------------+-------------------+--------------------+---+
|          cc_num|  first|    last|gender|              street|          city|state|  zip|    lat|     long|                 job|                dob|                 _id|age|
+----------------+-------+--------+------+--------------------+--------------+-----+-----+-------+---------+--------------------+-------------------+--------------------+---+
|3526015186182660|   Carl|   Gomez|     M|204 Cohen Meadow ...|Hathaway Pines|   CA|95233|38.1919|-120.3644|Data processing m...|1958-10-11 19:30:00|3526015186182660+...| 60|
|4170242670039985|Rebecca|Trujillo|     F|       242 Cody Pass|      Colstrip|   MT|59323|45.9344|-106.6368|          Air broker|1983-08-08 20:30:00|4170242670039985+...| 36|
+----------------+-------+--------+------+--------------------+--------------+-----+-----+-------+---------+-----------------

In [5]:
# Load transactions from source CSV file
raw_transaction_df = spark.read \
    .option('header', True) \
    .csv('{}/{}'.format(cfg.load.dir, cfg.load.transaction), schema = schema.transaction.schema)

In [6]:
# trans_date column from the source data is stored as timestamp with 00:00:00 time
# Convert trans_date to date and transform trans_time column to timestamp column.
transaction_df = raw_transaction_df \
    .withColumn("trans_date", split(col("trans_date"), "T")[0]) \
    .withColumn("trans_time", concat_ws(" ", col("trans_date"), col("trans_time"))) \
    .withColumn("trans_time", to_timestamp(col("trans_time"), "YYYY-MM-dd HH:mm:ss").cast(TimestampType()))

In [7]:
# Create UDF function from our Python function
# dist_udf = udf(earth_distance)

In [8]:
# Join transactions to customers to compute the distance
# project only columns that we want to export
processed_transactions_df = transaction_df.join(broadcast(customer_age_df), "cc_num") \
    .withColumn("distance", lit(round(udf_dist(col("lat"), col("long"), ("merch_lat"), col("merch_long")), 2))) \
    .select("cc_num", "trans_num", "trans_time", "category", "merchant", "amt", "merch_lat", "merch_long", "distance", "age", "is_fraud") \
    .withColumn("_id", col("trans_num")) \
    .cache()

In [9]:
# From transactions filter fraud transactions and nonfraud transactions in separate DataFrame objects
fraud_transactions_df = processed_transactions_df.where("is_fraud = 1")
nonfraud_transactions_df = processed_transactions_df.where("is_fraud = 0")

## Persist DataFrames

In [10]:
# Store customer data into MongoDB collection
# Because we defined _id column write performs "upser"
_ = mongo_save(customer_df, collection=cfg.db.customer)

In [11]:
# Store fraud transactions into MongoDB collection
# Writes are idempotent (upserts) due to _id column.
_ = mongo_save(fraud_transactions_df, collection=cfg.db.fraud)

In [12]:
# Store non-fraud transactions into MongoDB collection
# Writes are idempotent (upserts) due to _id column.
_ = mongo_save(nonfraud_transactions_df, collection=cfg.db.nonfraud)

In [13]:
# Store original transactions into MongoDB for study purposees
_ = mongo_save(transaction_df, collection=cfg.db.transaction)